## Creating region general score dataset
In this notebook, the predictions of all regional models are combined to get the predictions for all stations. Then, these predictions are used to compute the performance metrics of the regional models combined, which are comparable to the performance metrics computed for the general model. (Remember that the test stations used in regional and general scenarios are the same). Similarly, the general model predictions are separated to compute the performance metrics of the general model for each region, which are comparable to the regional models' performance metrics. The computed metrics are saved in **region_general_scores.csv**, where for each combination of `algorithm`, `region`, `input_combo`, and `dataset` a `region_metric` and a `general_metric` are provided for the regional models and the general model, respectively.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
os.chdir(Path.cwd().parents[0])

In [3]:
def load_predictions(algorithm, region, input_combo):
    
    if region is None:
        region = 'all_data'
        
    file_name = f"{algorithm}_{region.lower().replace(' ', '_')}_f{input_combo}.npz"
    pred = np.load(f'models/model_predictions/{file_name}')
    
    y_hat_train = pred['y_hat_train']
    y_hat_test = pred['y_hat_test']
    
    return y_hat_train, y_hat_test

In [4]:
def load_data(region):
    df = pd.read_csv('processed_data/et_data.csv')
    
    if region is not None:
        cond = df['region'] == region
        df = df.loc[cond].reset_index(drop=True)
        
    return df

In [5]:
def merge_predictions(data, y_hat_train, y_hat_test):
    df = data.copy()
    
    assert data.shape[0] == y_hat_train.shape[0] + y_hat_test.shape[0], "The predictions do not match the data"
    
    # Slicing train data
    cond = df['dataset'] == 'train'
    train_data = df.loc[cond]
    
    # Shuffling train data
    train_data = train_data.sample(frac=1, random_state=12).reset_index(drop=True)
    train_data['y_hat'] = y_hat_train
    
    # Slicing test data
    cond = df['dataset'] == 'test'
    test_data = df.loc[cond].copy()
    test_data['y_hat'] = y_hat_test
    
    df = pd.concat([train_data, test_data], ignore_index=True)
    df = df.sort_values(by=['st_num', 'year', 'month']).reset_index(drop=True)
    
    return df[['region', 'st_num', 'year', 'month', 'dataset', 'ET0', 'y_hat']]

In [6]:
score_fun_dict={
                 'MAE': mean_absolute_error,
                 'RMSE': lambda x, y: np.sqrt(mean_squared_error(x, y)),
                 'rRMSE': lambda x, y: np.sqrt(mean_squared_error(x, y)) / np.mean(x),
                 'NSE': r2_score}

In [7]:
def load_score(algorithm, region, input_combo, dataset, metric):
    
    df = pd.read_csv('models/model_results/model_scores.csv')
    
    if region is None:
        region = 'all_data'
        
    model_name = f"{algorithm}_{region.lower().replace(' ', '_')}_f{input_combo}"
    
    cond1 = df['model_name'] == model_name
    cond2 = df['dataset'] == dataset
    
    return df.loc[cond1 & cond2, metric].squeeze()

In [8]:
def get_cluster_general_score(algorithm, input_combo, dataset, score_fun_dict):
    
    regions_list = ['Mediterranean', 'Marmara', 'Black Sea', 'Central Anatolia',
                    'Southeastern Anatolia', 'Eastern Anatolia', 'Aegean']
    
    # Loading general model predictions
    df = load_data(None)
    y_hat_train, y_hat_test = load_predictions(algorithm, None, input_combo)
    general_preds = merge_predictions(df, y_hat_train, y_hat_test)
    
    # Slicing the defined dataset predictions
    cond = general_preds['dataset'] == dataset
    general_preds = general_preds.loc[cond]
    
    df_list = []
    
    combined_regional_preds = []
    
    for region in regions_list:
        
        result = pd.DataFrame(index=score_fun_dict.keys(), columns=['algorithm', 'region', 'input_combo',
                                                                 'dataset'])
        # Slicing the defined region
        cond = general_preds['region'] == region
        sliced_general_preds = general_preds.loc[cond]

        # Loading regional model predictions
        df = load_data(region)
        y_hat_train, y_hat_test = load_predictions(algorithm, region, input_combo)
        regional_preds = merge_predictions(df, y_hat_train, y_hat_test)

        # Slicing the defined dataset predictions
        cond = regional_preds['dataset'] == dataset
        regional_preds = regional_preds.loc[cond]

        for metric, fun in score_fun_dict.items():
            result.loc[metric, 'region_metric'] = fun(regional_preds['ET0'], regional_preds['y_hat'])
            result.loc[metric, 'general_metric'] = fun(sliced_general_preds['ET0'], sliced_general_preds['y_hat'])

        result['region'] = region
        
        df_list.append(result.reset_index().rename(columns={'index': 'metric'}))
        combined_regional_preds.append(regional_preds)
    
    # Combining metric dataframes of all regions
    df = pd.concat(df_list, ignore_index=True)
    
    # Combining regional model prediciotns
    combined_regional_preds = pd.concat(combined_regional_preds, ignore_index=True)
    
    # Computing the total scores for regional and general models
    result = pd.DataFrame(index=score_fun_dict.keys(), columns=['algorithm', 'region', 'input_combo',
                                                                 'dataset'])
    for metric, fun in score_fun_dict.items():
            result.loc[metric, 'region_metric'] = fun(combined_regional_preds['ET0'], combined_regional_preds['y_hat'])
            result.loc[metric, 'general_metric'] = fun(general_preds['ET0'], general_preds['y_hat'])
    
    result['region'] = 'all_data'
    
    df = pd.concat([df, result.reset_index().rename(columns={'index': 'metric'})], ignore_index=True)
    
    df['algorithm'] = algorithm
    df['input_combo'] = input_combo
    df['dataset'] = dataset
    
    return df

In [9]:
# Creating dataframe
df_list = []
algorithms = ['SVR', 'GPR', 'RF', 'Polynomial']

# for algorithm in algorithms:
#     for input_combo in range(1, 17):
#         for dataset in ['train', 'test']:
#             df_list.append(get_cluster_general_score(algorithm, input_combo, dataset, score_fun_dict))
            
# result = pd.concat(df_list, ignore_index=True)